In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
try: sys.path.index('/ml.dev/bin')
except: sys.path.append('/ml.dev/bin')

import numpy as n
import pandas as p
#mport Quandl as q
from Quandl import Quandl as q
import datetime as dd
from qoreliquid import *
import talib as talib
from matplotlib import pyplot as plt
from pylab import rcParams
%pylab inline
rcParams['figure.figsize'] = 20, 5

import oandapy

co = p.read_csv('config.csv', header=None)

env1=co.ix[0,1]
access_token1=co.ix[0,2]
oanda1 = oandapy.API(environment=env1, access_token=access_token1)

env2=co.ix[1,1]
access_token2=co.ix[1,2]
oanda2 = oandapy.API(environment=env2, access_token=access_token2)

acc = oanda2.get_accounts()['accounts']
accid = acc[0]['accountId']
#print 'using account: {0}'.format(accid)

In [ ]:
#oanda1.get_accounts()
targetSalaryPerSecond = (40e3 / (60*60*24*260))
asddf = p.DataFrame(oanda1.get_trades(558788)['trades'])#.set_index('instrument')#.ix[:,'side units'.split(' ')]
asddf['timeu'] = oq.oandaToTimestamp(asddf['time'])
asddf['timeage'] = time.time() - n.array(oq.oandaToTimestamp(asddf['time']))
asddf['targetSalaryPerSecond'] = targetSalaryPerSecond * asddf['timeage']
asddf.ix[:, 'time timeu timeage targetSalaryPerSecond'.split(' ')]

In [ ]:
import os
cmd = 'ls -t /tmp/patterns*'
fname = os.popen(cmd).read().strip().split('\n')[0]
dfh = {}
df = p.DataFrame()
if 1:
    dfu = p.DataFrame()
    for i in 'EUR_USD,GBP_USD,GBP_JPY,USD_CAD,EUR_AUD,USD_JPY,AUD_USD,AUD_JPY,CAD_JPY,EUR_CAD,EUR_CHF,EUR_GBP,NZD_JPY,NZD_USD,USD_CHF,CHF_JPY,USD_MXN'.split(','):
        dfu = dfu.combine_first(getc4(df, dfh, instrument=i))
    fname = '/tmp/patterns.dfu.%s.csv' % time.time()
    dfu.to_csv(fname)
else:
    dfu = p.read_csv(fname, index_col=0)
print dfu

In [ ]:
dfu['diff'] = n.abs(dfu['buy'].get_values() - dfu['sell'].get_values())
dfu['diffp'] = (dfu['diff'].get_values())/n.sum(dfu['diff'].get_values())
dfu['side'] = map(lambda x: 'buy' if x == 1 else 'sell', (n.array((dfu['buy'].get_values() - dfu['sell'].get_values()) > 0, dtype=int)))
dfu['sideBool'] = map(lambda x: 1 if x == 1 else -1, (n.array((dfu['buy'].get_values() - dfu['sell'].get_values()) > 0, dtype=int)))

dfu2 = dfu[dfu['diffp'] > (5.0/100)].sort('diff', ascending=False)
# recalculate percentages [diffp]
dfu2['diffp'] = (dfu2['diff'].get_values())/n.sum(dfu2['diff'].get_values())
with p.option_context('display.max_rows', 4000, 'display.max_columns', 4000, 'display.width', 1000000):
    print dfu2

In [ ]:
dfu2
#orders

In [ ]:
orders = dfu2.ix[:, 'diff side diffp sideBool'.split(' ')]
print 'Oanda orders:'
print orders
print

ps = """AUD/NZD
AUD/USD
EUR/CHF
EUR/GBP
EUR/JPY
EUR/NOK
EUR/SEK
EUR/USD
GBP/USD
NZD/USD
USD/CAD
USD/CHF
USD/CNH
USD/JPY
USD/RUB""".replace('/', '_')
#print ps

psdf = p.DataFrame(ps.split('\n'))
psdf['1'] = 1
psdf = psdf.set_index(0)
# list only 1broker pairs in the orderbook
print list(orders.index)
print psdf

oneBrokerOrderbook = psdf.ix[list(orders.index), :]
oneBrokerOrderbook[oneBrokerOrderbook['1'] == 1]
dfu3 = orders.ix[oneBrokerOrderbook[oneBrokerOrderbook['1'] == 1].index, :]

# recalculate percentages [diffp]
dfu3['diffp'] = (dfu3['diff'].get_values())/n.sum(dfu3['diff'].get_values())
print '1broker orders:'
print dfu3


In [ ]:
dfu3

In [ ]:
dryrun=True
#fu33 = rebalanceTrades(dfu2, oanda2, accid, dryrun=dryrun)
dfu33 = rebalanceTrades(dfu2, oanda1, 558788, dryrun=dryrun)

In [ ]:
cw()

In [ ]:
# source: https://mrjbq7.github.io/ta-lib/func.html
cl = n.random.random(200)
output = talib.SMA(cl)
#plot(cl)
df = p.DataFrame(cl)
df['sma'] = output
df.plot()

In [ ]:
#from talib import MA_Type
df['upper'], df['middle'], df['lower'] = talib.BBANDS(cl, matype=talib.MA_Type.T3)
df.plot()
df['momentum'] = talib.MOM(cl, timeperiod=5)
df.plot()

In [ ]:
bitcoinBalance = 0.40520
dfu33['amountp'] = bitcoinBalance * dfu33['diffp']
print dfu33

In [ ]:
oanda2.get_account(accid)['balance'] * 50

In [ ]:
lss = [1,2,3]
dir(lss)
lss.__contains__([1,2,3,4])

In [ ]:
list(orders.index)

In [ ]:
getc4(df, dfh, instrument='CAD_JPY', verbose=True, update=True)

In [ ]:
dfh['CAD_JPY'] = {}
getc4(df, dfh, instrument='CAD_JPY')

In [ ]:
d3 = dict(zip('M1 M5 M15 M30 H1 H4 D W M'.split(' '), [1,5,15,30,60,(60*4),(60*24), (60*24*5), (60*24)*5*4 ]))
print d3
print dfm.transpose().ix['USD_JPY','buy sell'.split(' ')]

In [ ]:
pa = ''
pa += 'USDJPY EURUSD'
df = getDataFromQuandlBNP(pa, 'EUR', noUpdate=True)
dd = normalizeme(df.bfill().ffill())
dd = sigmoidme(dd)
plt.plot(dd)
plt.show()

In [ ]:
#from charts.ipynb **copy**
def gettickerSymbol(s):
    hque = fetchURL('https://www.google.com/finance?q={0}'.format(s), mode='')
    try:
        hque = re.match(re.compile(r'.*\(([\w]+?:[\w]+?)\).*', re.S), hque).groups()[0]
        return hque
    except:
        return ''

def queryGoogleFinanceForTicker(st1):
    st1
    ty = []
    for i in st1:
        ty.append("{0}".format(gettickerSymbol(i)))
    print ty
    tt = ";".join(ty)
    tt = re.sub(re.compile(r':', re.S), '_', tt).split(';')
    print tt
    ty = []
    for i in tt:
        ty.append("GOOG/{0}".format(i))
    st1 = ty
    return st1
st1 = queryGoogleFinanceForTicker(['EURAUD'])

In [ ]:
#getDataFromQuandl(st1, plot=False, style='-', columns='Close', tail=100)
getDataFromQuandl(['CURRENCY:EUR'], plot=False, style='-', columns='Close', tail=100)
#ta.CDL2CROWS()